In [ ]:
import numpy as np
import tensorflow as tf
from sklearn import preprocessing
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers
import os
import random
import matplotlib.pyplot as plt
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
img_src =np.array([])
category = np.array([])
with open("/content/drive/MyDrive/train.csv", "r") as reader:
    reader.readline()
    for line in reader.readlines():
        line = line.split(",")
        line[1] = int(line[1].strip())
        img_src = np.append(img_src , line[0])
        category = np.append(category, line[1])
print(category)

[5. 4. 4. ... 0. 2. 2.]


In [ ]:
TRAINING_DIR = "/content/drive/MyDrive/географія/train_images_category"
training_datagen = ImageDataGenerator(
      rescale = 1./255,
      width_shift_range=0.2,
      height_shift_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest'
      )

train_generator = training_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=(300,300),
	class_mode='categorical',
  # shuffle = True,
  batch_size=32,
  seed = 1
)

CV_DIR = "/content/drive/MyDrive/географія/test_images_category"


test_datagen = ImageDataGenerator(
      rescale = 1./255)


test_generator = training_datagen.flow_from_directory(
	CV_DIR,
	target_size=(300,300),
	class_mode='categorical',
  # shuffle = True,

  batch_size=32
)


Found 2663 images belonging to 6 classes.
Found 121 images belonging to 6 classes.


In [ ]:
pre_trained_model = tf.keras.applications.EfficientNetV2B3(input_shape = (300, 300, 3),
                                include_top = False,
                                weights = "imagenet")

In [ ]:
print(pre_trained_model.layers)

[<keras.engine.input_layer.InputLayer object at 0x7fb7406af1f0>, <keras.layers.preprocessing.image_preprocessing.Rescaling object at 0x7fb740609990>, <keras.layers.preprocessing.normalization.Normalization object at 0x7fb6706a6ec0>, <keras.layers.convolutional.conv2d.Conv2D object at 0x7fb68aff6860>, <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7fb73819c5b0>, <keras.layers.core.activation.Activation object at 0x7fb66fcb9180>, <keras.layers.convolutional.conv2d.Conv2D object at 0x7fb66fcb9960>, <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7fb66fcbad10>, <keras.layers.core.activation.Activation object at 0x7fb73819fbb0>, <keras.layers.convolutional.conv2d.Conv2D object at 0x7fb66fcbb190>, <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7fb66fcbb100>, <keras.layers.core.activation.Activation object at 0x7fb66fcbae60>, <keras.layers.regularization.dropout.Dropout object at 0x7fb66fd46e60>, <k

In [ ]:
pre_trained_model.summary()

Model: "efficientnetv2-b3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 300, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_1 (Rescaling)        (None, 300, 300, 3)  0           ['input_2[0][0]']                
                                                                                                  
 normalization_1 (Normalization  (None, 300, 300, 3)  0          ['rescaling_1[0][0]']            
 )                                                                                                
                                                                                  

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import Model


x = tf.keras.layers.GlobalAveragePooling2D()(pre_trained_model.output)
x = tf.keras.layers.Dropout(0.50)(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(6, activation='softmax')(x)

model = Model(pre_trained_model.input, x)

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 300, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_1 (Rescaling)        (None, 300, 300, 3)  0           ['input_2[0][0]']                
                                                                                                  
 normalization_1 (Normalization  (None, 300, 300, 3)  0          ['rescaling_1[0][0]']            
 )                                                                                                
                                                                                            

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.000001)

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint("bestfin111123.keras", monitor='val_accuracy', save_best_only=True, mode='auto')

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(category),
                                        y = category
                                    )
class_weights = dict(zip(np.unique(category), class_weights))

In [ ]:



model.compile(loss = "categorical_crossentropy", optimizer="nadam", metrics=['accuracy'])
history = model.fit(train_generator,
validation_data=(test_generator), class_weight = class_weights,
epochs=60,batch_size = 32, callbacks=[reduce_lr,checkpoint])

Epoch 1/60
84/84 [==============================] - 199s 1s/step - loss: 1.6680 - accuracy: 0.3323 - val_loss: 2.0053 - val_accuracy: 0.1736 - lr: 0.0010
Epoch 2/60
84/84 [==============================] - 96s 1s/step - loss: 1.2319 - accuracy: 0.5456 - val_loss: 2.3432 - val_accuracy: 0.1570 - lr: 0.0010
Epoch 3/60
84/84 [==============================] - 97s 1s/step - loss: 0.9540 - accuracy: 0.6545 - val_loss: 2.2083 - val_accuracy: 0.1653 - lr: 0.0010
Epoch 4/60
84/84 [==============================] - 98s 1s/step - loss: 0.7977 - accuracy: 0.6936 - val_loss: 2.1496 - val_accuracy: 0.1653 - lr: 0.0010
Epoch 5/60
84/84 [==============================] - 101s 1s/step - loss: 0.6988 - accuracy: 0.7424 - val_loss: 1.6726 - val_accuracy: 0.4628 - lr: 0.0010
Epoch 6/60
84/84 [==============================] - 97s 1s/step - loss: 0.6481 - accuracy: 0.7657 - val_loss: 1.9791 - val_accuracy: 0.4050 - lr: 0.0010
Epoch 7/60
84/84 [==============================] - 98s 1s/step - loss: 0.5344 -

In [ ]:
model.save("best.keras")

NameError: ignored